## Currency Conversion 
    - using API

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool, InjectedToolArg
from typing import Annotated
from langchain_core.messages import HumanMessage
import requests

from dotenv import load_dotenv
load_dotenv()

True

### Tool Creation

In [12]:
# Tool Create

@tool     #1 tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches currency conversion factor between a give base currency and target currency
    """

    url = f"https://v6.exchangerate-api.com/v6/d46fd5b48e4b82f8969cf035/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    
    return response.json()



@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [3]:
# Testing First tool
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756252801,
 'time_last_update_utc': 'Wed, 27 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1756339201,
 'time_next_update_utc': 'Thu, 28 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.6692}

In [13]:
# Checking arguments of the Secong tool
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [14]:
# Testing the second tool (basically it's a function, so just calling a function using .invoke since it's a llm tool now!)
convert.invoke({'base_currency_value':10, 'conversion_rate':87.6051})

876.0509999999999

### Tool Binding

In [15]:
# tool binding
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [16]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert]) # binding all the tools, it's a list we can add remove and define which to use and which to remove!

In [8]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR')] # creating a list to store conversion

In [17]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [18]:
ai_message = llm_with_tools.invoke(messages) # making AI read and respond to Human message

In [19]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': 'd050290d-65c9-4924-b80b-1c49bcdb5d78',
  'type': 'tool_call'}]

In [20]:
for tool_call in ai_message.tool_calls:
    # Execute 1st tool and get value of (conversion rate)
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        print(tool_message1)
    # Execute 2nd tool using conversion rate from tool 1


content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1756252801, "time_last_update_utc": "Wed, 27 Aug 2025 00:00:01 +0000", "time_next_update_unix": 1756339201, "time_next_update_utc": "Thu, 28 Aug 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 87.6692}' name='get_conversion_factor' tool_call_id='d050290d-65c9-4924-b80b-1c49bcdb5d78'
